>[Load in Colab](#scrollTo=lm-MDLOPmwBa)

>[Initiate](#scrollTo=9OBnxzC1m1tc)

>[Prostate](#scrollTo=PQ2l2jDef9MT)

>[Create Cross folds](#scrollTo=q-fTOxiBZcm7)

>[Train](#scrollTo=dEO65i01p-bt)

>>[Train Functions](#scrollTo=zDtjJeVkBQGd)

>>[UNET](#scrollTo=nUQFtc_GoE0s)

>>[MNet](#scrollTo=vwW3vyUBoQAK)

>>[UNet3+](#scrollTo=8ADLHu-Y3l80)



# Load in Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Prostate_MRI_Detection_3D_Model"

/content/drive/MyDrive/Prostate_MRI_Detection_3D_Model


In [3]:
!pip install -r requirement.txt
!pip install monai
!pip install dicom2nifti

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 88.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Initiate

In [4]:
#prostate
HOME_DIR =       "./"
DATA_DIR =       "./PROSTATEx_masks/Files/prostate/"
OUT_DIR =        "./cross_results/pheripheral/"
SLICED_OUT_DIR = "./data/sliced/prostate/"
#AUG_OUT_DIR = "./data/augmented_test/prostate/"
FOLD_DIR = "./cross/pheripheral/"

In [5]:
import os
from glob import glob
import shutil
from tqdm import tqdm
import dicom2nifti
import numpy as np
import nibabel as nib
import pandas as pd
from monai.utils import first

import json
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import os

In [6]:
import torch
import nibabel as nib

import elasticdeform 
from skimage.util import random_noise
import scipy
from scipy.ndimage import affine_transform

import torchvision.transforms.functional as TF
import nibabel as nib
#import torchio as tio

import numpy as np
from scipy import ndimage
from scipy.ndimage import shift
import random

In [7]:
from monai.utils import first
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
from monai.losses import DiceLoss
from tqdm import tqdm
from utils.train import train

# Pheripheral 
Create 16 images 
need to run if data/sliced/pz_maskks aren't available
need to run proste first

In [8]:
def get_data_path(path):
  f = open( path + 'config.json')
  jdata = json.load(f)
  f.close()
  return jdata["path"]

In [9]:
if not os.path.exists(SLICED_OUT_DIR + "mask_pz"):
    os.makedirs(SLICED_OUT_DIR + "mask_pz")

In [10]:
def makeLabelNo(name):
  numb= name.split("-")[1]
  li=[80,81,83,85,86,87,88,89,90,91,93,96,98]
  if(int(numb) in li):
    return 'ProstateX-' + numb[1:]
  if (int(numb) in [202,203]):
    return 'VOLUME-'+ numb
  return name

In [11]:
df = pd.read_csv(DATA_DIR +'image_list.csv')

In [12]:
data = [{'image': DATA_DIR + 'Images/' + x + ".nii.gz", "label":DATA_DIR + 'mask_pz/' + makeLabelNo(x.split("_")[0])+'_pz' + ".nii.gz"  } for x in df['T2']]

In [13]:
slice_size = 16

In [14]:
from utils.preprocessing import create_same_slice_zone

create_same_slice_zone(data, slice_size , SLICED_OUT_DIR + 'config_pz.json')


1/204
2/204
3/204
4/204
5/204
6/204
7/204
8/204
9/204
10/204
11/204
12/204
13/204
14/204
15/204
16/204
17/204
18/204
19/204
20/204
21/204
22/204
23/204
24/204
25/204
26/204
27/204
28/204
29/204
30/204
31/204
32/204
33/204
34/204
35/204
36/204
37/204
38/204
39/204
40/204
41/204
42/204
43/204
44/204
45/204
46/204
47/204
48/204
49/204
50/204
51/204
52/204
53/204
54/204
55/204
56/204
57/204
58/204
59/204
60/204
61/204
62/204
63/204
64/204
65/204
66/204
67/204
68/204
69/204
70/204
71/204
72/204
73/204
74/204
75/204
76/204
77/204
78/204
79/204
80/204
81/204
82/204
83/204
84/204
85/204
86/204
87/204
88/204
89/204
90/204
91/204
92/204
93/204
94/204
95/204
96/204
97/204
98/204
99/204
100/204
101/204
102/204
103/204
104/204
105/204
106/204
107/204
108/204
109/204
110/204
111/204
112/204
113/204
114/204
115/204
116/204
117/204
118/204
119/204
120/204
121/204
122/204
123/204
124/204
125/204
126/204
127/204
128/204
129/204
130/204
131/204
132/204
133/204
134/204
135/204
136/204
137/204
138/204
139/

In [15]:
data = get_data_path(SLICED_OUT_DIR)

# Create Cross folds

need to run if 5 folds weren't created

In [16]:
def get_data_path(path):
  f = open( path + 'config_pz.json')
  jdata = json.load(f)
  f.close()
  return jdata["path"]

In [17]:
def save_to_json(data, path):
  with open(path, 'w') as fp:
    json.dump(data, fp)

In [18]:
data = get_data_path(SLICED_OUT_DIR)

In [19]:
from sklearn.model_selection import KFold

k=5
kfold = KFold(n_splits=k)

for index, (train_indices, test_indices) in enumerate(kfold.split(data)):
  name = "fold"+str(index)
  

  if not os.path.exists(FOLD_DIR + name ):
        os.makedirs(FOLD_DIR + name)

  train_paths = []
  for index in train_indices:
    train_paths.append( data[index])

  save_to_json({"paths": train_paths}, FOLD_DIR + name + "/train_config.json")

  test_paths = []
  for index in test_indices:
    test_paths.append( data[index])

  save_to_json({ "paths": test_paths}, FOLD_DIR + name + "/test_config.json")
 

# Train

## Train Functions

In [20]:
from utils.transform import transform
import os.path
import torch

def updateLogs(path, data):
    f = open(path,'a')
    f.write(data)
    f.close()

def get_data_path(path):
  f = open( path )
  jdata = json.load(f)
  f.close()
  return jdata["paths"]

def Cross_Val(model_class,  model_args, loss_function, load_from, fold_dir, model_dir , epochs=100, test_interval= 5,start_from= 1, device=torch.device("cpu") , start_fold=0, num_folds=5):

  for index in range(start_fold , num_folds):

    fold = 'fold'+str(index)
    
    train_data = get_data_path(fold_dir + fold +"/train_config.json")
    test_data = get_data_path(fold_dir + fold +"/test_config.json")

    pixdim =(1.5, 1.5, 1.0)
    a_min=0
    a_max=500
    spatial_size= [128, 128,16] #[384, 384,18]

    train_loader = transform(train_data, a_min, a_max, spatial_size, pixdim)
    test_loader = transform(test_data, a_min, a_max, spatial_size, pixdim)

    model_fold_dir = model_dir + fold+'/'

    if not os.path.exists(model_fold_dir):
      os.makedirs(model_fold_dir)

    data_in = [train_loader, test_loader]

    model = model_class(**model_args).to(device)
    
    path =  model_dir + fold +"/best_metric_model.pth"

    if (os.path.exists(path)):
          model.load_state_dict(torch.load(
              os.path.join(path), map_location=device))
          print("Loaded") 

    
    optimizer = torch.optim.Adam(model.parameters(), 1e-3, weight_decay=1e-5, amsgrad=True)

    
    print(f"{fold} starts_______________________\n")
    updateLogs(os.path.join(model_dir, "logs.txt"), f"{fold} starts_______________________\n")
    train(model, data_in, loss_function, optimizer, epochs, model_fold_dir, test_interval= 1,start_from= 1, load_from = model_fold_dir)
    updateLogs(os.path.join(model_dir, "logs.txt"), f"{fold} ends _______________________\n")

In [21]:
import torch 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)

## UNET

In [23]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm


model_dir = OUT_DIR +"unet-test_2/"
model_args = {
    'num_res_units': 2,
    'channels': (64, 128, 256, 512),
    'out_channels': 1,
    'in_channels': 1,
    'norm': Norm.BATCH,
    'spatial_dims': 3,
    'strides': (2, 2, 2),
}

Cross_Val(UNet, model_args, loss_function, fold_dir=FOLD_DIR, model_dir= model_dir, 
          epochs=2, test_interval= 1,start_from= 1, device=device, load_from = model_dir, start_fold=1, num_folds=5)

/usr/local/lib/python3.9/dist-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


fold1 starts_______________________

----------
epoch 1/2


/usr/local/lib/python3.9/dist-packages/monai/losses/dice.py:144: UserWarning: single channel prediction, `to_onehot_y=True` ignored.
  warnings.warn("single channel prediction, `to_onehot_y=True` ignored.")


1/163, Train_loss: 0.9120 Train_dice: 0.0880
2/163, Train_loss: 0.9180 Train_dice: 0.0820


KeyboardInterrupt: ignored

## MNet

In [24]:
from Models.ModifiedNet import MNet

model_dir = OUT_DIR +"mnet-test/"
model_args = {
   
}

Cross_Val(MNet, model_args, loss_function, fold_dir=FOLD_DIR, model_dir= model_dir, 
          epochs=2, test_interval= 1,start_from= 1, device=device, load_from = model_dir, start_fold=1, num_folds=5)

fold1 starts_______________________

----------
epoch 1/2


KeyboardInterrupt: ignored

## UNet3+

In [ ]:
from Models.Unet_TriplePlus import U_net_TriplePlus

model_dir = OUT_DIR +"unet_3plus-test/"
model_args = {
    'in_channel' :1, 'out_channel':1
}

Cross_Val( U_net_TriplePlus, model_args, loss_function, fold_dir=FOLD_DIR, model_dir= model_dir, 
          epochs=2, test_interval= 1,start_from= 1, device=device, load_from = model_dir, start_fold=1, num_folds=5)

fold1 starts_______________________

----------
epoch 1/2
